In [2]:
import xarray as xr
import pandas as pd
import cdsapi
import os

In [3]:
def obtener_rango_fechas(archivo_nc):
    # Abrir el archivo .nc usando xarray
    ds = xr.open_dataset(archivo_nc)

    # Obtener la variable de tiempo (generalmente llamada 'time')
    if 'valid_time' in ds.variables:
        tiempos = ds['valid_time'].values

        # Convertir los tiempos a formato datetime
        fechas = [pd.to_datetime(t) for t in tiempos]

        # Obtener la fecha mínima y máxima
        fecha_min = fechas[0].strftime('%Y-%m-%d')
        fecha_max = fechas[-1].strftime('%Y-%m-%d')

        return fecha_min, fecha_max
    else:
        return None, None

def cambiar_nombre(archivo_nc):
    fecha_min, fecha_max = obtener_rango_fechas(archivo_nc)
    if fecha_min and fecha_max:
        #nuevo_nombre = f"{fecha_min}_a_{fecha_max}.nc"
        nuevo_nombre = f"{fecha_min}_a_{fecha_max}_adicional.nc"
        nuevo_archivo_completo = os.path.join(nuevo_nombre)
        try:
            # Intentar renombrar el archivo
            os.rename(archivo_nc, nuevo_archivo_completo)
            print(f"Archivo {archivo_nc} renombrado a {nuevo_nombre}")
            return nuevo_archivo_completo
        except PermissionError:
            print(f"Archivo {archivo_nc} en uso. No se puede renombrar en este momento.")
            return archivo_nc
        except Exception as e:
            print(f"Ocurrió un error al renombrar {archivo_nc}: {e}")
            return archivo_nc
    else:
        print(f"No se pudo obtener el rango de fechas para {archivo}")
        return archivo_nc
        
#Extracción por lotes        
def extraerCopernicus(year_min, year_max):
    for year in range(year_max, year_min,-1):
        for month in range(12,0,-1):
         dataset = "reanalysis-era5-single-levels"
         request = {
        'product_type': ['ensemble_mean'],
        'variable': ['total_precipitation', 'skin_temperature', 'evaporation', 'runoff', 'snowfall', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'high_vegetation_cover', 'low_vegetation_cover', 'type_of_high_vegetation', 'type_of_low_vegetation'],
        'year': [str(year)],
        'month': [str(month)],
        'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
        'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
        'data_format': 'netcdf',
        'download_format': 'unarchived',
        'area': [40.5425, -2.255, 38.1739, 0.5665] ##Cuenca hidrográfica del Jucar +/-
        }
        client = cdsapi.Client()
        file_name = client.retrieve(dataset, request).download()
        #file_name = cambiar_nombre(file_name)

def construirCSV (csv_output):
    agg_funcs = {
    'total_precipitation': 'sum',
    'skin_temperature': 'mean',
    'evaporation': 'sum',
    'runoff': 'sum',
    'snowfall': 'sum',
    'soil_water_l1': 'sum',
    'soil_water_l2': 'sum',
    'soil_water_l3': 'sum',
    'soil_water_l4': 'sum',
    'high_vegetation_cover': 'mean',
    'low_vegetation_cover': 'mean',
    'type_high_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'type_low_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan
    }

    # Bucle sobre cada archivo NetCDF
    for idx, archivo in enumerate(os.listdir()):
        if archivo.endswith('.nc'):
            print(f"Procesando: {archivo}")
            archivo_nc = os.path.join(archivo)
            
            # Abrir el archivo NetCDF y convertir a DataFrame
            ds = xr.open_dataset(archivo_nc)
            df = ds.to_dataframe().reset_index()
    
            # Filtrar y renombrar columnas
            df = df[['valid_time', 'latitude', 'longitude', 'tp', 'skt', 'e', 'ro', 'sf',
                     'swvl1', 'swvl2', 'swvl3', 'swvl4', 'cvh', 'cvl', 'tvh', 'tvl']]
            df = df.rename(columns={
                'valid_time': 'date',
                'tp': 'total_precipitation',
                'skt': 'skin_temperature',
                'e': 'evaporation',
                'ro': 'runoff',
                'sf': 'snowfall',
                'swvl1': 'soil_water_l1',
                'swvl2': 'soil_water_l2',
                'swvl3': 'soil_water_l3',
                'swvl4': 'soil_water_l4',
                'cvh': 'high_vegetation_cover',
                'cvl': 'low_vegetation_cover',
                'tvh': 'type_high_vegetation',
                'tvl': 'type_low_vegetation'
            })
    
            # Convertir la columna 'date' a formato de fecha
            df['date'] = pd.to_datetime(df['date']).dt.date
    
            # Realizar las agregaciones
            df = df.groupby(['latitude', 'longitude', 'date']).agg(agg_funcs).reset_index()
    
            # Escribir en CSV. Si es la primera vez, incluir el encabezado, de lo contrario, hacer append sin encabezado.
            if idx == 0:
                df.to_csv(csv_output, mode='w', index=False, header=True)
            else:
                df.to_csv(csv_output, mode='a', index=False, header=False)
    
            # Cerrar el dataset para liberar memoria
            ds.close()
def extraccionCompletaArchivos (year_min, year_max,):
    extraerCopernicus(year_min, year_max)
    construirCSV ('df_copernicus.csv')

#Extracción parcial
def extraccionUnicaCopernicus (year, month):
    dataset = "reanalysis-era5-single-levels"
    request = {
    'product_type': ['ensemble_mean'],
    'variable': ['total_precipitation', 'skin_temperature', 'evaporation', 'runoff', 'snowfall', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'high_vegetation_cover', 'low_vegetation_cover', 'type_of_high_vegetation', 'type_of_low_vegetation'],
    'year': [str(year)],
    'month': [str(month)],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
    'data_format': 'netcdf',
    'download_format': 'unarchived',
    'area': [40.5425, -2.255, 38.1739, 0.5665] ##Cuenca hidrográfica del Jucar +/-
    }
    client = cdsapi.Client()
    file_name = client.retrieve(dataset, request).download()
    new_name = cambiar_nombre(file_name)
    return new_name
def ampliarCSV(archivo_csv, archivo_nc):
    agg_funcs = {
    'total_precipitation': 'sum',
    'skin_temperature': 'mean',
    'evaporation': 'sum',
    'runoff': 'sum',
    'snowfall': 'sum',
    'soil_water_l1': 'sum',
    'soil_water_l2': 'sum',
    'soil_water_l3': 'sum',
    'soil_water_l4': 'sum',
    'high_vegetation_cover': 'mean',
    'low_vegetation_cover': 'mean',
    'type_high_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'type_low_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan}
    ds = xr.open_dataset(archivo_nc)
    df = ds.to_dataframe().reset_index()

    # Filtrar y renombrar columnas
    df = df[['valid_time', 'latitude', 'longitude', 'tp', 'skt', 'e', 'ro', 'sf',
             'swvl1', 'swvl2', 'swvl3', 'swvl4', 'cvh', 'cvl', 'tvh', 'tvl']]
    df = df.rename(columns={
        'valid_time': 'date',
        'tp': 'total_precipitation',
        'skt': 'skin_temperature',
        'e': 'evaporation',
        'ro': 'runoff',
        'sf': 'snowfall',
        'swvl1': 'soil_water_l1',
        'swvl2': 'soil_water_l2',
        'swvl3': 'soil_water_l3',
        'swvl4': 'soil_water_l4',
        'cvh': 'high_vegetation_cover',
        'cvl': 'low_vegetation_cover',
        'tvh': 'type_high_vegetation',
        'tvl': 'type_low_vegetation'
    })

    # Convertir la columna 'date' a formato de fecha
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Realizar las agregaciones
    df = df.groupby(['latitude', 'longitude', 'date']).agg(agg_funcs).reset_index()
    df.to_csv(archivo_csv, mode='a', index=False, header=False)
def extraccionParcialArchivos (añoMax,añoMin, mesMax = 12, mesMin = 1):
    años = range(añoMax, añoMin - 1 ,-1)
    meses = range(mesMax, mesMin - 1,-1)
    for year in años: 
        for month in meses:
            archivo_nc = extraccionUnicaCopernicus(year = year, month = month)
            #ampliarCSV (archivo_csv = 'df_copernicus.csv', archivo_nc = archivo_nc)
            print('Enlazados datos de', archivo_nc, 'a','df_copernicus.csv')



In [4]:
for year in range(2024,1959,-1):
    for month in range(12,0,-1):
        dataset = "reanalysis-era5-single-levels"
        request = {
        'product_type': ['ensemble_mean'],
        'variable': ['total_precipitation', 'skin_temperature', 'evaporation', 'runoff', 'snowfall', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'high_vegetation_cover', 'low_vegetation_cover', 'type_of_high_vegetation', 'type_of_low_vegetation'],
        'year': [str(2024)],
        'month': [str(month)],
        'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
        'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
        'data_format': 'netcdf',
        'download_format': 'unarchived',
        'area': [40.7433, -1.1988, 40.2756, 0.8996] ##Cuenca hidrográfica del Jucar +/-
        }
        client = cdsapi.Client()
        file_name = client.retrieve(dataset, request).download()
        file_name = cambiar_nombre(file_name)

2024-09-30 19:26:48,419 WARNING [2024-09-30T17:26:48.438084] You are using a deprecated API endpoint. If you are using cdsapi, please upgrade to the latest version.
2024-09-30 19:26:48,435 INFO Request ID is f6f42c9c-56cd-463f-a96e-39deb28cb3e3
2024-09-30 19:26:48,524 INFO status has been updated to accepted
2024-09-30 19:35:11,133 INFO status has been updated to failed


HTTPError: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/f6f42c9c-56cd-463f-a96e-39deb28cb3e3/results
The job has failed.
The job failed with: ValueError

In [9]:
for year in range(1965, 1960,-1):
    if year == 1965:
        meses = range(6,0, -1)
    else:
        meses = range(12,0, -1)
    for month in meses:
        dataset = "reanalysis-era5-single-levels"
        request = {
            'product_type': ['ensemble_mean'],
            'variable': ['total_precipitation', 'skin_temperature', 'evaporation', 'runoff', 'snowfall', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'high_vegetation_cover', 'low_vegetation_cover', 'type_of_high_vegetation', 'type_of_low_vegetation'],
            'year': [str(year)],
            'month': [str(month)],
            'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
            'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
            'data_format': 'netcdf',
            'download_format': 'unarchived',
            'area': [40.5425, -2.255, 38.1739, 0.5665] ##Cuenca hidrográfica del Jucar +/-
        }
        
        client = cdsapi.Client()
        file_name = client.retrieve(dataset, request).download()
        fecha_min, fecha_max = obtener_rango_fechas(file_name)
        if fecha_min and fecha_max:
            nuevo_nombre = f"{fecha_min}_a_{fecha_max}.nc"
            nuevo_archivo_completo = os.path.join(nuevo_nombre)

            try:
                # Intentar renombrar el archivo
                os.rename(file_name, nuevo_archivo_completo)
                print(f"Archivo {file_name} renombrado a {nuevo_nombre}")
            except PermissionError:
                print(f"Archivo {file_name} en uso. No se puede renombrar en este momento.")
            except Exception as e:
                print(f"Ocurrió un error al renombrar {file_name}: {e}")
        else:
            print(f"No se pudo obtener el rango de fechas para {archivo}")

2024-09-22 09:14:46,566 WARNING [2024-09-22T07:14:47.015717] You are using a deprecated API endpoint. If you are using cdsapi, please upgrade to the latest version.
2024-09-22 09:14:46,568 INFO Request ID is 498b7570-b4ee-4bf5-b36e-9e58b8eefd10
2024-09-22 09:14:46,647 INFO status has been updated to accepted
2024-09-22 09:14:48,449 INFO status has been updated to successful


8d9f33e5f015292e6aa306c8924e2110.nc:   0%|          | 0.00/268k [00:00<?, ?B/s]

Archivo 8d9f33e5f015292e6aa306c8924e2110.nc renombrado a 1965-06-01_a_1965-06-30.nc


2024-09-22 09:14:57,372 WARNING [2024-09-22T07:14:57.830419] You are using a deprecated API endpoint. If you are using cdsapi, please upgrade to the latest version.
2024-09-22 09:14:57,388 INFO Request ID is 2feb7a18-fc13-4bf4-a70f-534981e26e4c
2024-09-22 09:14:57,474 INFO status has been updated to accepted
2024-09-22 09:14:59,045 INFO status has been updated to running
2024-09-22 09:17:54,211 INFO status has been updated to successful


8b508562dff36d53a312e489637af3b0.nc:   0%|          | 0.00/272k [00:00<?, ?B/s]

...

2471a30a92d4740b2117f1a35d7d2fb5.nc:   0%|          | 0.00/259k [00:00<?, ?B/s]

Archivo 2471a30a92d4740b2117f1a35d7d2fb5.nc renombrado a 1961-02-01_a_1961-02-28.nc


2024-09-22 11:40:59,842 WARNING [2024-09-22T09:41:00.037681] You are using a deprecated API endpoint. If you are using cdsapi, please upgrade to the latest version.
2024-09-22 11:40:59,842 INFO Request ID is 589d57f8-ee70-4eb3-9bab-487fa4a3ad87
2024-09-22 11:40:59,949 INFO status has been updated to accepted
2024-09-22 11:41:01,516 INFO status has been updated to running
2024-09-22 11:42:55,625 INFO status has been updated to successful


16c3659ab30e1946aa0bc999c275e421.nc:   0%|          | 0.00/282k [00:00<?, ?B/s]

Archivo 16c3659ab30e1946aa0bc999c275e421.nc renombrado a 1961-01-01_a_1961-01-31.nc


In [10]:
csv_output = 'df_copernicus.csv'  # Nombre del archivo CSV de salida
agg_funcs = {
    'total_precipitation': 'sum',
    'skin_temperature': 'mean',
    'evaporation': 'sum',
    'runoff': 'sum',
    'snowfall': 'sum',
    'soil_water_l1': 'sum',
    'soil_water_l2': 'sum',
    'soil_water_l3': 'sum',
    'soil_water_l4': 'sum',
    'high_vegetation_cover': 'mean',
    'low_vegetation_cover': 'mean',
    'type_high_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'type_low_vegetation': lambda x: x.mode()[0] if not x.mode().empty else np.nan
}

# Bucle sobre cada archivo NetCDF
for idx, archivo in enumerate(os.listdir()):
    if archivo.endswith('.nc'):
        print(f"Procesando: {archivo}")
        archivo_nc = os.path.join(archivo)
        
        # Abrir el archivo NetCDF y convertir a DataFrame
        ds = xr.open_dataset(archivo_nc)
        df = ds.to_dataframe().reset_index()

        # Filtrar y renombrar columnas
        df = df[['valid_time', 'latitude', 'longitude', 'tp', 'skt', 'e', 'ro', 'sf',
                 'swvl1', 'swvl2', 'swvl3', 'swvl4', 'cvh', 'cvl', 'tvh', 'tvl']]
        df = df.rename(columns={
            'valid_time': 'date',
            'tp': 'total_precipitation',
            'skt': 'skin_temperature',
            'e': 'evaporation',
            'ro': 'runoff',
            'sf': 'snowfall',
            'swvl1': 'soil_water_l1',
            'swvl2': 'soil_water_l2',
            'swvl3': 'soil_water_l3',
            'swvl4': 'soil_water_l4',
            'cvh': 'high_vegetation_cover',
            'cvl': 'low_vegetation_cover',
            'tvh': 'type_high_vegetation',
            'tvl': 'type_low_vegetation'
        })

        # Convertir la columna 'date' a formato de fecha
        df['date'] = pd.to_datetime(df['date']).dt.date

        # Realizar las agregaciones
        df = df.groupby(['latitude', 'longitude', 'date']).agg(agg_funcs).reset_index()

        # Escribir en CSV. Si es la primera vez, incluir el encabezado, de lo contrario, hacer append sin encabezado.
        if idx == 0:
            df.to_csv(csv_output, mode='w', index=False, header=True)
        else:
            df.to_csv(csv_output, mode='a', index=False, header=False)

        # Cerrar el dataset para liberar memoria
        ds.close()


Procesando: 1961-01-01_a_1961-01-31.nc
Procesando: 1961-02-01_a_1961-02-28.nc
Procesando: 1961-03-01_a_1961-03-31.nc
Procesando: 1961-04-01_a_1961-04-30.nc
Procesando: 1961-05-01_a_1961-05-31.nc
Procesando: 1961-06-01_a_1961-06-30.nc
Procesando: 1961-07-01_a_1961-07-31.nc
Procesando: 1961-08-01_a_1961-08-31.nc
Procesando: 1961-09-01_a_1961-09-30.nc
Procesando: 1961-10-01_a_1961-10-31.nc
Procesando: 1961-11-01_a_1961-11-30.nc
Procesando: 1961-12-01_a_1961-12-31.nc
Procesando: 1962-01-01_a_1962-01-31.nc
Procesando: 1962-02-01_a_1962-02-28.nc
Procesando: 1962-03-01_a_1962-03-31.nc
Procesando: 1962-04-01_a_1962-04-30.nc
Procesando: 1962-05-01_a_1962-05-31.nc
Procesando: 1962-06-01_a_1962-06-30.nc
Procesando: 1962-07-01_a_1962-07-31.nc
Procesando: 1962-08-01_a_1962-08-31.nc
Procesando: 1962-09-01_a_1962-09-30.nc
Procesando: 1962-10-01_a_1962-10-31.nc
Procesando: 1962-11-01_a_1962-11-30.nc
Procesando: 1962-12-01_a_1962-12-31.nc
Procesando: 1963-01-01_a_1963-01-31.nc
Procesando: 1963-02-01_a_